#  Testing Thorlabs DCx camera

In [1]:
from instrumental import instrument, list_instruments
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pylab as p




In [19]:

paramsets = list_instruments(module='cameras')
# print(paramsets)
print(paramsets[0])
daq = instrument('bla')

<ParamSet[UC480_Camera] serial=b'4103175850' model=b'C1284R13C' id=1>


Exception: Instrument with alias `bla` not found in config file

## Grab one image and display

In [ ]:

img = daq.grab_image()
#adjust brightness
img_adj = img.sum(axis = 2)
img_adj = cv2.equalizeHist(np.uint8(img_adj))
    
    
# plt.imshow(img_adj, aspect="auto")
# plt.show()
# fig = plt.figure()
ax = plt.subplot(111)
im = ax.imshow(img_adj, cmap='gray')

##  Start video stream

In [ ]:
# ax = plt.subplot(111)
import matplotlib.animation as animation

# cv2.namedWindow( 'video', cv2.WINDOW_AUTOSIZE );

img = daq.grab_image(timeout='1s', copy=True)
img_adj = img.sum(axis = 2)
img_adj = cv2.equalizeHist(np.uint8(img_adj))


fig = plt.figure()
ax = plt.subplot(111)
img_disp = ax.imshow(img_adj, cmap='gray')
plt.pause(0.1)

daq.start_live_video(framerate="10 hertz")
while True:
    daq.wait_for_frame(timeout=None)
    img = daq.latest_frame(copy=True)
    img_adj = img.sum(axis = 2)
    img_adj = cv2.equalizeHist(np.uint8(img_adj))
    img_disp.set_data(img_adj)
    fig.canvas.draw()
    plt.pause(0.001)
    

In [ ]:
import matplotlib.animation as animation

# cv2.namedWindow( 'video', cv2.WINDOW_AUTOSIZE );

def animate(i):
    img = daq.latest_frame(copy=True)
    img_adj = img.sum(axis = 2)
    img_adj = cv2.equalizeHist(np.uint8(img_adj))
    ax.clear()
    ax.imshow(img_adj, cmap='gray')
    print('bla')
    

fig = plt.figure()
ax = plt.subplot(111)

daq.start_live_video(framerate="10 hertz")
ani = animation.FuncAnimation(fig, animate, interval=10)
plt.show()


    

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

class LivePlotNotebook(object):
    """
    Live plot using %matplotlib notebook in jupyter notebook
    
    Usage:
    ```
    import time
    liveplot = LivePlotNotebook()
    x=np.random.random((10,))
    for i in range(10):
        time.sleep(1)
        liveplot.update(
            x=x+np.random.random(x.shape)/10,
            actions=np.random.randint(0, 3, size=(10,))
        )
    ```
    
    url:
    """

    def __init__(self):
        %matplotlib notebook
        fig,ax = plt.subplots(1,1)
        
        ax.plot([0]*20, label='price')
        ax.plot([1]*20, [1]*20, 'o', ms=12,c='gray', label='hold')
        ax.plot([0]*20, [0]*20, '^', ms=12,c='blue', label='buy' )
        ax.plot([0]*20, [0]*20, 'v', ms=12,c='red', label='sell')
        
        ax.set_xlim(0,1)
        ax.set_ylim(0,1)
        ax.legend()
        ax.set_xlabel('timesteps')
        ax.grid()
        ax.set_title('actions')
        
        self.ax = ax
        self.fig = fig

    def update(self, x, actions):             
        # update price
        line = self.ax.lines[0]
        line.set_xdata(range(len(x)))
        line.set_ydata(x)
        
        # update action plots
        for i, line in enumerate(self.ax.lines[1:]):
            line.set_xdata(np.argwhere(actions==i).T)
            line.set_ydata(x[actions==i])
            line.set_marker(['o','^','v'][i])

        # update limits
        self.ax.set_xlim(0, len(actions))
        self.ax.set_ylim(x.min(), x.max())

        self.fig.canvas.draw()
        
# Test
import time
liveplot = LivePlotNotebook()
x=np.random.random((10,))
for i in range(10):
    time.sleep(1)
    liveplot.update(
        x=x+np.random.random(x.shape)/10,
        actions=np.random.randint(0, 3, size=(10,))
)